In [1]:
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
from sklearn.neural_network import MLPRegressor

In [3]:
model_name = "multi_layer_perceptron"
display_name = "Multi-Layer \nPerceptron"

In [4]:
y_test = pd.read_csv('y_test.csv')
file = open("Selection Methods","rb")
sel_methods = np.load(file)

In [5]:
def get_best_model(method, args, _cv, is_sample):
    sample_string = ''
    if (is_sample):
        sample_string = '_sample'
    y_train = pd.read_csv('y_train'+ sample_string + '.csv')    
    method_string = method
    if (method == ''):
        method_string = "no feature selection"
    print('\n\nRunning cross fold validation for',display_name,'with',method_string,'dataset')
    x_train = pd.read_csv('x_train' + method + sample_string + '.csv').values
    x_test = pd.read_csv('x_test' + method + '.csv').values
    pipe = Pipeline([('clf', MLPRegressor())])
    grid = GridSearchCV(pipe, args, cv = _cv, verbose = 1, scoring = 'r2', n_jobs=6)
    grid.fit(x_train, y_train.values.ravel())
    print('\nBest score for',display_name,'with',method_string,"dataset:",grid.best_score_)
    best_args = grid.best_estimator_.get_params()
    print('\nBest hyperparameters for',display_name,'with',method_string,'dataset:')
    for arg in best_args:
        print(arg,":",best_args[arg])
    y_test_predict = grid.predict(x_test)
    mse = mean_squared_error(y_test,y_test_predict)
    mae = mean_absolute_error(y_test,y_test_predict)
    r2 = r2_score(y_test,y_test_predict)
    print('\nPrediction scores for',display_name,'using',method_string,':')
    print('Mean Squared error:',mse)
    print('Mean Absolute error:',mae)
    print('R^2:',r2)
    return grid.best_estimator_,grid.best_score_,mse,mae,r2

In [6]:
def save_model(model,method):
    with open('CrimeGUI/Models/'+model_name+method,'wb') as file:
        pickle.dump(model,file)

In [7]:
print(sel_methods)

['_f_regression' '_chi2' '_adaboost' '_equal_crime_and_business'
 '_all_business']


<h3>F-Regression feature selected dataset<h3>

In [28]:
is_sample = True
num_folds = 5
args = {'clf__activation':('identity','logistic','tanh','relu'),
        'clf__alpha':(0.000001,0.00001,0.0001,0.001,0.01,0.1),
        'clf__batch_size':(arange(100, 300, 100)),
        'clf__hidden_layer_sizes':((100,),(200,),(300,)),
        'clf__learning_rate':('constant', 'invscaling', 'adaptive'),
        'clf__max_iter':(arange(100,300,100)),
        'clf__momentum':(arange(0.7,0.9,0.1)),
        'clf__power_t':(0.001,0.01,0.1),
        'clf__random_state':([7]),
        'clf__solver':('lbfgs','logistic','adam'),
        'clf__tol':(0.0001,0.001,0.01)
       }

In [29]:
f_reg_model, f_reg_train_score, f_reg_mse, f_reg_mae, f_reg_r2 = get_best_model(sel_methods[0], args, num_folds, is_sample)



Running cross fold validation for Multi-Layer 
Perceptron with _f_regression dataset
Fitting 5 folds for each of 69984 candidates, totalling 349920 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.3s
[Parallel(n_jobs=6)]: Done 212 tasks      | elapsed:   17.8s
[Parallel(n_jobs=6)]: Done 519 tasks      | elapsed:   43.3s
[Parallel(n_jobs=6)]: Done 967 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 1556 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 2270 tasks      | elapsed:  3.1min
[Parallel(n_jobs=6)]: Done 3021 tasks      | elapsed:  4.4min
[Parallel(n_jobs=6)]: Done 3836 tasks      | elapsed:  5.9min
[Parallel(n_jobs=6)]: Done 4758 tasks      | elapsed:  7.6min
[Parallel(n_jobs=6)]: Done 5708 tasks      | elapsed: 10.4min
[Parallel(n_jobs=6)]: Done 6758 tasks      | elapsed: 13.7min
[Parallel(n_jobs=6)]: Done 8023 tasks      | elapsed: 16.2min
[Parallel(n_jobs=6)]: Done 9572 tasks      | elapsed: 18.1min
[Parallel(n_jobs=6)]: Done 11133 tasks      | elapsed: 21.1min
[Parallel(n_jobs=6)]: Done 12696 tasks      | elapsed: 24.9m


Best score for Multi-Layer 
Perceptron with _f_regression dataset: 0.7533626543777094

Best hyperparameters for Multi-Layer 
Perceptron with _f_regression dataset:
memory : None
steps : [('clf', MLPRegressor(activation='logistic', alpha=0.1, batch_size=100,
             hidden_layer_sizes=(200,), max_iter=100, momentum=0.7,
             power_t=0.001, random_state=7, solver='lbfgs', tol=0.01))]
verbose : False
clf : MLPRegressor(activation='logistic', alpha=0.1, batch_size=100,
             hidden_layer_sizes=(200,), max_iter=100, momentum=0.7,
             power_t=0.001, random_state=7, solver='lbfgs', tol=0.01)
clf__activation : logistic
clf__alpha : 0.1
clf__batch_size : 100
clf__beta_1 : 0.9
clf__beta_2 : 0.999
clf__early_stopping : False
clf__epsilon : 1e-08
clf__hidden_layer_sizes : (200,)
clf__learning_rate : constant
clf__learning_rate_init : 0.001
clf__max_fun : 15000
clf__max_iter : 100
clf__momentum : 0.7
clf__n_iter_no_change : 10
clf__nesterovs_momentum : True
clf__power_

In [ ]:
save_model(f_reg_model,sel_methods[0])

<h3>Chi-Squared feature selected dataset<h3>

In [8]:
# folds reduced to 3 for initial exploration due to excessive processing time
is_sample = True
num_folds = 3
args = {'clf__activation':('identity','logistic','tanh','relu'),
        'clf__alpha':(0.000001,0.00001,0.0001,0.001,0.01,0.1),
        'clf__batch_size':(arange(100, 300, 100)),
        'clf__hidden_layer_sizes':((100,),(200,),(300,)),
        'clf__learning_rate':('constant', 'invscaling', 'adaptive'),
        'clf__max_iter':(arange(100,300,100)),
        'clf__momentum':(arange(0.7,0.9,0.1)),
        'clf__power_t':(0.001,0.01,0.1),
        'clf__random_state':([7]),
        'clf__solver':('lbfgs','logistic','adam'),
        'clf__tol':(0.0001,0.001,0.01)
       }

In [9]:
chi2_model, chi2_train_score, chi2_mse, chi2_mae, chi2_r2 = get_best_model(sel_methods[1], args, num_folds, is_sample)



Running cross fold validation for Multi-Layer 
Perceptron with _chi2 dataset
Fitting 3 folds for each of 69984 candidates, totalling 209952 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   21.5s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   48.4s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  3.5min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  5.1min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  7.0min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed:  9.4min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed: 11.3min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed: 13.0min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 15.8min
[Parallel(n_jobs=6)]: Done 8438 tasks      | elapsed: 19.9min
[Parallel(n_jobs=6)]: Done 9788 tasks      | elapsed: 22.5min
[Parallel(n_jobs=6)]: Done 11238 tasks      | elapsed: 25.8mi


Best score for Multi-Layer 
Perceptron with _chi2 dataset: 0.7466949779592725

Best hyperparameters for Multi-Layer 
Perceptron with _chi2 dataset:
memory : None
steps : [('clf', MLPRegressor(alpha=1e-05, batch_size=100, hidden_layer_sizes=(200,),
             momentum=0.7, power_t=0.001, random_state=7))]
verbose : False
clf : MLPRegressor(alpha=1e-05, batch_size=100, hidden_layer_sizes=(200,),
             momentum=0.7, power_t=0.001, random_state=7)
clf__activation : relu
clf__alpha : 1e-05
clf__batch_size : 100
clf__beta_1 : 0.9
clf__beta_2 : 0.999
clf__early_stopping : False
clf__epsilon : 1e-08
clf__hidden_layer_sizes : (200,)
clf__learning_rate : constant
clf__learning_rate_init : 0.001
clf__max_fun : 15000
clf__max_iter : 200
clf__momentum : 0.7
clf__n_iter_no_change : 10
clf__nesterovs_momentum : True
clf__power_t : 0.001
clf__random_state : 7
clf__shuffle : True
clf__solver : adam
clf__tol : 0.0001
clf__validation_fraction : 0.1
clf__verbose : False
clf__warm_start : False



In [ ]:
save_model(chi2_model,sel_methods[1])

<h3>AdaBoost feature selected dataset<h3>

In [8]:
is_sample = True
num_folds = 3
args = {'clf__activation':('identity','logistic','tanh','relu'),
        'clf__alpha':(0.000001,0.00001,0.0001,0.001,0.01,0.1),
        'clf__batch_size':(arange(100, 300, 100)),
        'clf__hidden_layer_sizes':((100,),(200,),(300,)),
        'clf__learning_rate':('constant', 'invscaling', 'adaptive'),
        'clf__max_iter':(arange(100,300,100)),
        'clf__momentum':(arange(0.7,0.9,0.1)),
        'clf__power_t':(0.001,0.01,0.1),
        'clf__random_state':([7]),
        'clf__solver':('lbfgs','logistic','adam'),
        'clf__tol':(0.0001,0.001,0.01)
       }

In [9]:
ada_model, ada_train_score, ada_mse, ada_mae, ada_r2 = get_best_model(sel_methods[2], args, num_folds, is_sample)



Running cross fold validation for Multi-Layer 
Perceptron with _adaboost dataset
Fitting 3 folds for each of 69984 candidates, totalling 209952 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   24.7s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   53.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  2.4min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  3.7min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  5.5min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  7.9min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed: 11.3min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed: 13.5min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed: 15.4min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 18.6min
[Parallel(n_jobs=6)]: Done 8438 tasks      | elapsed: 23.9min
[Parallel(n_jobs=6)]: Done 9788 tasks      | elapsed: 27.2min
[Parallel(n_jobs=6)]: Done 11238 tasks      | elapsed: 30.9mi


Best score for Multi-Layer 
Perceptron with _adaboost dataset: 0.7531055438512761

Best hyperparameters for Multi-Layer 
Perceptron with _adaboost dataset:
memory : None
steps : [('clf', MLPRegressor(alpha=0.1, batch_size=100, momentum=0.7, power_t=0.001,
             random_state=7))]
verbose : False
clf : MLPRegressor(alpha=0.1, batch_size=100, momentum=0.7, power_t=0.001,
             random_state=7)
clf__activation : relu
clf__alpha : 0.1
clf__batch_size : 100
clf__beta_1 : 0.9
clf__beta_2 : 0.999
clf__early_stopping : False
clf__epsilon : 1e-08
clf__hidden_layer_sizes : (100,)
clf__learning_rate : constant
clf__learning_rate_init : 0.001
clf__max_fun : 15000
clf__max_iter : 200
clf__momentum : 0.7
clf__n_iter_no_change : 10
clf__nesterovs_momentum : True
clf__power_t : 0.001
clf__random_state : 7
clf__shuffle : True
clf__solver : adam
clf__tol : 0.0001
clf__validation_fraction : 0.1
clf__verbose : False
clf__warm_start : False

Prediction scores for Multi-Layer 
Perceptron using 

In [ ]:
save_model(ada_model,sel_methods[2])

<h3>Equal business and crime feature selected dataset<h3>

In [10]:
is_sample = True
num_folds = 3
args = {'clf__activation':('identity','logistic','tanh','relu'),
        'clf__alpha':(0.000001,0.00001,0.0001,0.001,0.01,0.1),
        'clf__batch_size':(arange(100, 300, 100)),
        'clf__hidden_layer_sizes':((100,),(200,),(300,)),
        'clf__learning_rate':('constant', 'invscaling', 'adaptive'),
        'clf__max_iter':(arange(100,300,100)),
        'clf__momentum':(arange(0.7,0.9,0.1)),
        'clf__power_t':(0.001,0.01,0.1),
        'clf__random_state':([7]),
        'clf__solver':('lbfgs','logistic','adam'),
        'clf__tol':(0.0001,0.001,0.01)
       }

In [ ]:
equal_model, equal_train_score, equal_mse, equal_mae, equal_r2 = get_best_model(sel_methods[3], args, num_folds, is_sample)



Running cross fold validation for Multi-Layer 
Perceptron with _equal_crime_and_business dataset
Fitting 3 folds for each of 69984 candidates, totalling 209952 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   20.4s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   46.3s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  3.4min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  5.3min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  7.7min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed: 10.6min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed: 12.8min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed: 15.0min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed: 18.5min
[Parallel(n_jobs=6)]: Done 8438 tasks      | elapsed: 23.7min
[Parallel(n_jobs=6)]: Done 9788 tasks      | elapsed: 27.1min
[Parallel(n_jobs=6)]: Done 11238 tasks      | elapsed: 30.7mi

In [ ]:
save_model(equal_model,sel_methods[3])

<h3>Business only feature selected dataset<h3>

In [ ]:
is_sample = True
num_folds = 3
args = {'clf__activation':('identity','logistic','tanh','relu'),
        'clf__alpha':(0.000001,0.00001,0.0001,0.001,0.01,0.1),
        'clf__batch_size':(arange(100, 300, 100)),
        'clf__hidden_layer_sizes':((100,),(200,),(300,)),
        'clf__learning_rate':('constant', 'invscaling', 'adaptive'),
        'clf__max_iter':(arange(100,300,100)),
        'clf__momentum':(arange(0.7,0.9,0.1)),
        'clf__power_t':(0.001,0.01,0.1),
        'clf__random_state':([7]),
        'clf__solver':('lbfgs','logistic','adam'),
        'clf__tol':(0.0001,0.001,0.01)
       }

In [ ]:
bus_model, bus_train_score, bus_mse, bus_mae, bus_r2 = get_best_model(sel_methods[4], args, num_folds, is_sample)

In [ ]:
save_model(bus_model,sel_methods[4])

<h3>Visualise scores<h3>

In [ ]:
training_scores = [f_reg_train_score, chi2_train_score, ada_train_score, equal_train_score, bus_train_score]
mse_scores = [f_reg_mse, chi2_mse, ada_mse, equal_mse, bus_mse]
mae_scores = [f_reg_mae, chi2_mae, ada_mae, equal_mae, bus_mae]
r2_scores = [f_reg_r2, chi2_r2, ada_r2, equal_r2, bus_r2]
datasets = ["F-Regression","Chi-Squared","AdaBoost","Equal Business\nand Crime","All Business"]
y_select = np.arange(len(datasets))

In [ ]:
plt.figure(figsize = (10,10))
plt.xticks(y_select,datasets)
plt.title(display_name + " tree training score")
plt.xlabel("Dataset")
plt.ylabel("Training score (r-squared)")
plt.bar(y_select, training_scores)

In [ ]:
plt.figure(figsize = (10,10))
plt.xticks(y_select,datasets)
plt.title(display_name + " testing mean-squared error")
plt.xlabel("Dataset")
plt.ylabel("Mean-squared error")
plt.bar(y_select, mse_scores)

In [ ]:
plt.figure(figsize = (10,10))
plt.xticks(y_select,datasets)
plt.title(display_name + " testing mean-absolute error")
plt.xlabel("Dataset")
plt.ylabel("Mean absolute error")
plt.bar(y_select, mae_scores)

In [ ]:
plt.figure(figsize = (10,10))
plt.xticks(y_select,datasets)
plt.title(display_name + " testing r-squared scores")
plt.xlabel("Dataset")
plt.ylabel("R-Squared Score")
plt.bar(y_select, r2_scores)